# Sheikhspeare

## AI chatbot that speaks in Shakespearean English

<a href="https://colab.research.google.com/github/cybardev/Sheikhspeare/blob/main/sheikhspeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Define the model name and load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

In [4]:
def raw_generator(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Define a sample text for summarization
sample_text = """
How are you doing today?
"""

# Summarize the sample text using the pre-trained model (without fine-tuning)
pre_finetuned_summary = raw_generator(sample_text)
print("Summary before fine-tuning:", pre_finetuned_summary)

In [ ]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset, which contains articles and summaries
dataset = load_dataset("Roudranil/shakespearean-and-modern-english-conversational-dataset", split="train")

In [7]:
# Split the dataset into training and testing subsets
dataset_split = dataset.train_test_split(test_size=0.1)

# Further reduce the training set size for faster testing during development
small_train_dataset = dataset_split['train'].train_test_split(test_size=0.99)['train']
eval_dataset = dataset_split['test']

In [8]:
def preprocess_function(examples):
  # Extract the articles from the dataset
  inputs = [doc for doc in examples['translated_dialog']]

  # Tokenize the articles (inputs) with padding and truncation to a max length of 512
  model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

  # Tokenize the summaries (labels) using the target tokenizer context
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['og_response'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")

  # Attach the tokenized summaries as labels to the model inputs
  model_inputs["labels"] = labels["input_ids"]

  # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
  model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

  return model_inputs

In [ ]:
# Tokenize the small training dataset
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)

# Tokenize the evaluation dataset
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    evaluation_strategy="epoch",         # Evaluate the model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Regularization to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=3,                  # Number of training epochs
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs"                 # Directory for storing training logs
)

In [11]:
from transformers import Seq2SeqTrainer

# Create the trainer object
trainer = Seq2SeqTrainer(
    model=model,                         # The model to be trained
    args=training_args,                  # The training arguments defined earlier
    train_dataset=tokenized_train_dataset,  # The tokenized training dataset
    eval_dataset=tokenized_eval_dataset,    # The tokenized evaluation dataset
    tokenizer=tokenizer                  # The tokenizer to handle input and output
)

In [ ]:
# Let's train
trainer.train()

In [ ]:
# Evaluate the model on the evaluation dataset
metrics = trainer.evaluate()

# Print the evaluation metrics
print(metrics)

In [14]:
def tuned_generator(text):
  # Tokenize the input text and move it to the correct device
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)

  # Generate the summary using the fine-tuned model
  summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)

  # Decode the generated summary back into text and return it
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(tuned_generator("""
How are you doing today?
"""
))